In [245]:
import os 
import time
import json
import nltk
import string
import pickle
import datetime
import pickle
import pandas as pd
from os import listdir
import re
from sklearn.metrics import f1_score, recall_score, precision_score


In [3]:
import numpy as np
import pandas as pd
import xml.etree.ElementTree as xml

In [4]:
from moviepy.editor import *
from tqdm.notebook import tqdm
from collections import Counter

from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

strptime = datetime.datetime.strptime

In [420]:
mentions_path = 'coref/summaries_mentions_first_story.pickle'
mentions = pickle.load(open(mentions_path, 'rb'))
print('support', mentions['s01e07']['support'][:2], '..')
print('marked', mentions['s01e07']['marked'][:2], '..')
print('characters', mentions['s01e07']['characters'][:2], '..')


support ['Synopsis When four members of a family are brutally murdered in a suburban house, except the two daughters who are the sole survivors, the nightshift is called', 'At first, Gil Grissom and his entire team suspect a bizarre cult murder, but the evidence tells an even more disturbing story, when they discover the motive behind the massacre'] ..
marked ['Synopsis When four members of a family are brutally murdered in a suburban house , except the two daughters who are the sole survivors , the nightshift is called', 'At first , Gil #Grissom and his entire team suspect a bizarre cult murder , but the evidence tells an even more disturbing story , when they discover the motive behind the massacre'] ..
characters [[], ['Grissom']] ..


In [421]:
path_screenplays_scenes='coref/csi-corpus/screenplay_summarization/scene_level_n_aspects'
all_chars = []
ep_characters = {}

eps=[]

for ep in listdir(path_screenplays_scenes):
    #print(ep)
    annotated_scenes=pd.read_csv(path_screenplays_scenes+'/'+ep)
    eps.append(ep.split('.csv')[0])
    




In [390]:
print(eps)
print(eps[-7])



['s03e12', 's05e13', 's01e20', 's05e10', 's01e08', 's03e11', 's04e14', 's05e22', 's03e19', 's04e23', 's05e21', 's02e06', 's04e10', 's05e08', 's03e05', 's04e22', 's04e09', 's05e06', 's01e13', 's02e01', 's05e17', 's01e07', 's04e21', 's05e12', 's02e15', 's04e12', 's03e03', 's04e05', 's02e10', 's02e09', 's05e05', 's05e03', 's02e04', 's04e15', 's01e23', 's03e21', 's04e06', 's03e08', 's01e19']
s02e04


In [133]:
df=pd.DataFrame()
df['content'] = 
print('yo')

yo


In [51]:
ps = PorterStemmer()
ls = LancasterStemmer()
do_stem = True


In [14]:
ps.stem('kills')

'kill'

In [ ]:
def compute_similarity(content, summary_line, method):
    if method[0] == 'tfidf':
        weights = method[1]
        return sum(weights[w] if w in weights else 0 for w in set(content.split()) if w in summary_line.split())
    if method[0] == 'tfidf_log':
        weights = method[1]
        dn = np.log(len(content.split() + summary_line.split()) + 1) if len(content.strip().split()) > 0 else 1
        return sum(weights[w] if w in weights else 0 for w in set(content.split()) if w in summary_line.split())/dn
    if method[0] == 'tfidf_log_2':
        weights = method[1]
        n = sum(1 if w in weights else 0 for w in set(content.split()) if w in summary_line.split())
        dn = np.log(len(content.split() + summary_line.split()) - n + 1) 
        dn = dn if dn > 0 else 1
        return sum(weights[w] if w in weights else 0 for w in set(content.split()) if w in summary_line.split())/dn
    if method[0] == 'tfidf_log_3':
        weights = method[1]
        n = sum(1 if w in weights else 0 for w in set(content.split()) if w in summary_line.split())
        dn = np.log10(len(content.split() + summary_line.split()) - n + 1) 
        dn = dn if dn > 0 else 1
        return sum(weights[w] if w in weights else 0 for w in set(content.split()) if w in summary_line.split())/dn
    if method[0] == 'tfidf_sqrt':
        weights = method[1]
        n = sum(1 if w in weights else 0 for w in set(content.split()) if w in summary_line.split())
        dn = np.sqrt(len(content.split() + summary_line.split()) - n + 1) 
        dn = dn if dn > 0 else 1
        return sum(weights[w] if w in weights else 0 for w in set(content.split()) if w in summary_line.split())/dn
    elif method[0] == 'count':
        return len([w for w in set(content.split()) if w in summary_line.split()])
    elif method[0] == 'count_2':
        dn = np.log(len(content.split() + summary_line.split()) + 1) 
        dn = dn if dn > 0 else 1
        return len([w for w in set(content.split()) if w in summary_line.split()]) / dn

In [440]:
f1_scores=[]

#eps=['s02e04']


#eps=['s03e12', 's05e13', 's01e20', 's05e10', 's01e08', 's04e14', 's05e22', 's03e19', 's04e23', 's05e21', 's02e06', 's04e10', 's05e08', 's03e05', 's04e22', 's05e06', 's01e13', 's02e01', 's05e17', 's01e07', 's04e21', 's05e12', 's02e15', 's04e12', 's03e03', 's04e05', 's02e10', 's02e09', 's05e05', 's05e03', 's04e15', 's01e23', 's03e21', 's04e06', 's03e08', 's01e19']

for ep in eps:
    method_used=('tfidf',tfidf_weights)
    annotated_scenes=pd.read_csv(path_screenplays_scenes+'/'+ep+'.csv')
    nb_positive=len(annotated_scenes[annotated_scenes['in_summary'] == 1])
    vectorizer = TfidfVectorizer(min_df=2, stop_words='english')
    corpus = [' '.join(ps.stem(w) for w in word_tokenize(s) if w.lower() not in vectorizer.get_stop_words()) if do_stem else s for s in annotated_scenes['scene_text'].values]
    corpus = [s.translate(str.maketrans('', '', string.punctuation)).replace('  ', ' ').lower() for s in corpus]
    #print(len(corpus))
    #print(len(summaries))
    vectorizer.fit(corpus)
    corpus_tf_idf = vectorizer.transform(corpus)
    word_count = Counter([w for s in corpus for w in s.split()])
    ', '.join(sorted(vectorizer.get_stop_words()))
    tfidf_weights = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
    summaries_raw = [s for s in mentions[ep]['support']]
    summaries = [' '.join(ps.stem(w) for w in word_tokenize(s) if w.lower() not in vectorizer.get_stop_words()) if do_stem else s for s in summaries_raw]
    summaries = [s.translate(str.maketrans('', '', string.punctuation)).replace('  ', ' ') for s in summaries]
    similarity_matrix = np.zeros((len(corpus), len(summaries)))
    for i, content in tqdm(enumerate(corpus), total=len(corpus)):
        for j, s in enumerate(summaries):
            #print(i)
            #print(j)
            similarity_matrix[i, j] = compute_similarity(content, s, method=method_used)
    unrolled_sm = [(similarity_matrix[i, j], i, j)  for i in range(len(corpus)) for j in range(len(summaries))]
    best_matches = sorted(unrolled_sm, key=lambda x: -x[0])
    score = {}
    #print(j, summaries_raw[j])
    for (v, scene, syn) in best_matches:
        if scene not in score : score[scene] = []
        score[scene].append(syn)
        
        
    for s in score:
        #print(max(score[s]))
        #print(sum(score[s]))
        #score[s] = sum(score[s])
        score[s] = max(score[s])

        
    reses = sorted(score.items(), key=lambda x: -x[1])
    scenes=[i[0] for i in reses[0:nb_positive]]
    #print(reses[0])

    #print(syn, summaries_raw[syn])
    
    
    annotated_scenes['prediction'] = annotated_scenes['scene_id'].apply(lambda x: 1 if x in scenes else 0)
    #annotated_scenes['synopses'] = annotated_scenes['prediction'].apply(lambda x: summaries_raw[syn]  if x==1 else 0)
    annotated_scenes['aspect?'] = annotated_scenes['aspects'].apply(lambda x: 0 if x == 'None' else 1)
    print(method_used[0])
    annotated_scenes.to_csv(ep+'_'+method_used[0]+'_similarity.csv')
    
    #print('Episode')
    #print(ep)
    print(f1_score(annotated_scenes.in_summary, annotated_scenes.prediction))
    f1_scores.append(f1_score(annotated_scenes.in_summary, annotated_scenes.prediction))
    
    


tfidf
0.36363636363636365



tfidf
0.7777777777777778



tfidf
0.6153846153846154



tfidf
0.3333333333333333



tfidf
0.45454545454545453


...


tfidf
0.36363636363636365



tfidf
0.47058823529411764



tfidf
0.25



tfidf
0.5



tfidf
0.36363636363636365


In [473]:
print(len(f1_scores))
print(np.mean(f1_scores))
print(f1_scores[15])

method_used=('tfidf',tfidf_weights)
ep=eps[15]
print(ep)

#print(df)
print(df['aspect?'][df['aspect?'] == 1].sum() )
print(df['in_summary'][df['in_summary'] == 1].sum() )


df_asp=pd.DataFrame(columns=['ep', 'asp_sum', 'asp_nosum', 'noasp_sum', 'nb_in_sum','nb_aspect'])
for ep in eps:
    df=pd.read_csv(ep+'_'+method_used[0]+'_similarity.csv')  
    in_summary=df.loc[df['in_summary'] == 1]
    asp=df.loc[df['aspect?'] == 1]
    asp_sum=df.loc[(df['aspect?'] == 1) & (df['in_summary'] == 1)]
    asp_nosum=df.loc[(df['aspect?'] == 1) & (df['in_summary'] == 0)]
    noasp_sum=df.loc[(df['aspect?'] == 0) & (df['in_summary'] == 1)]
    episode= pd.DataFrame(data= {'ep': [ep], 'asp_sum': [len(asp_sum)], 'asp_nosum': [len(asp_nosum)], 'noasp_sum':[len(noasp_sum)],'nb_in_sum':[len(in_summary)],'nb_aspect':[len(asp)]})
    #print(episode)
    df_asp=df_asp.append(episode,ignore_index=True)

df_asp.loc['Total']= df_asp.sum()  
df_asp['ep']['Total']=''
print(df_asp)







39
0.410673527076242
0.20000000000000004
s04e22
12
11
           ep asp_sum asp_nosum noasp_sum nb_in_sum nb_aspect
0      s03e12       9         1         2        11        10
1      s05e13       9         0         0         9         9
2      s01e20      12         2         1        13        14
3      s05e10      12         3         0        12        15
4      s01e08      11         4         0        11        15
5      s03e11       8         2         1         9        10
6      s04e14      10         0         5        15        10
7      s05e22       7         1         0         7         8
8      s03e19      18         0         2        20        18
9      s04e23      10         2         3        13        12
10     s05e21       9         2         4        13        11
11     s02e06      11         0         1        12        11
12     s04e10      14         3         2        16        17
13     s05e08      10         0         1        11        10
14     s03e05   

In [433]:
aspects =['Crime scene', 'Victim', 'Death cause', 'Perpetrator', 'Evidence', 'Motive']
#aspects =['Crime scene']

for aspect in aspects:
    df_aspect=pd.DataFrame(columns=['scene_id', 'scene_text', 'in_summary', 'aspects',
       'prediction','ep','aspect_mentionned'])
    for ep in eps:
        df=pd.read_csv(ep+'_'+method_used[0]+'_similarity.csv')
        match=df[['scene_id', 'scene_text', 'in_summary', 'aspects',
       'prediction']][df.isin([aspect]).aspects]
        #print(match)
        match['ep']=ep
        #match['aspect_mentionned']=match.scene_text.apply(lambda lst: 1 if x for x in lst in aspect.lower() else 0 )
        #match['aspect_mentionned']=match.scene_text.apply(lambda x: 1 if x.contains(aspect.lower()) else 0 )
        #any(aspect.lower() in x for x in lst)
        #match['aspect_mentionned']=match[['aspects']][match.scene_text.str.contains('Camera')]='True'
        match['aspect_mentionned']= np.where(match.scene_text.str.contains(aspect.lower()), 1, 0)
        
        
        
        df_aspect=df_aspect.append(match,ignore_index=True)
        
    #df_aspect.to_csv(aspect+'.csv')
    #print(df_aspect.ep[4])
    #print(df_aspect.scene_text[4])
    #print(df_aspect.in_summary[4])
    print(aspect)
    print(df_aspect['aspect_mentionned'].value_counts())
    #print(df_aspect['scene_text'])
    #print('Recall',recall_score(df_aspect.in_summary.astype(int), df_aspect.prediction.astype(int)))
    #print('Precision',precision_score(df_aspect.in_summary.astype(int), df_aspect.prediction.astype(int)))
    #print('-----------------------------------------------------------')

        


Crime scene
0    22
1     5
Name: aspect_mentionned, dtype: int64
Victim
0    26
1     9
Name: aspect_mentionned, dtype: int64
Death cause
0    32
Name: aspect_mentionned, dtype: int64
Perpetrator
0    57
1     1
Name: aspect_mentionned, dtype: int64
Evidence
0    153
1     25
Name: aspect_mentionned, dtype: int64
Motive
0    42
1     3
Name: aspect_mentionned, dtype: int64


In [54]:
for ep in eps:
    df=pd.read_csv(ep+'_'+method_used[0]+'_similarity.csv')
    print(df)

970


In [55]:
word_count

Counter({'camera': 24,
         'open': 17,
         'collin': 39,
         'mail': 2,
         'box': 3,
         'sprinkler': 3,
         'push': 5,
         'slowli': 1,
         'bunni': 2,
         'lawn': 5,
         'decor': 2,
         'near': 2,
         'shut': 3,
         'moment': 3,
         'door': 26,
         'woman': 12,
         'scream': 3,
         'tinacollin': 46,
         'help': 10,
         'tina': 37,
         'run': 8,
         'hous': 26,
         'oh': 6,
         'god': 3,
         'somebodi': 6,
         'street': 1,
         'lung': 1,
         'neighbor': 5,
         's': 122,
         'light': 9,
         'have': 2,
         'turn': 26,
         'reach': 10,
         'start': 13,
         'pound': 1,
         'emerg': 1,
         'vehicl': 3,
         'offic': 9,
         'car': 9,
         'surround': 1,
         'grissom': 298,
         'drive': 1,
         'scene': 13,
         'dispatch': 2,
         'radio': 1,
         'park': 1,
         'exit':

In [22]:
corpus_tf_idf.shape

(36, 271)

In [56]:
print(len(vectorizer.get_stop_words()))


318


'a, about, above, across, after, afterwards, again, against, all, almost, alone, along, already, also, although, always, am, among, amongst, amoungst, amount, an, and, another, any, anyhow, anyone, anything, anyway, anywhere, are, around, as, at, back, be, became, because, become, becomes, becoming, been, before, beforehand, behind, being, below, beside, besides, between, beyond, bill, both, bottom, but, by, call, can, cannot, cant, co, con, could, couldnt, cry, de, describe, detail, do, done, down, due, during, each, eg, eight, either, eleven, else, elsewhere, empty, enough, etc, even, ever, every, everyone, everything, everywhere, except, few, fifteen, fifty, fill, find, fire, first, five, for, former, formerly, forty, found, four, from, front, full, further, get, give, go, had, has, hasnt, have, he, hence, her, here, hereafter, hereby, herein, hereupon, hers, herself, him, himself, his, how, however, hundred, i, ie, if, in, inc, indeed, interest, into, is, it, its, itself, keep, las

In [58]:
tfidf_weights['camera']

2.466337068793427

In [59]:
tfidf_weights['man']

3.5649493574615367

In [60]:
tfidf_weights['hallway']

2.3121863889661687

In [135]:

#summaries_raw = [' '.join(w[1:] if w[0] in ['@', '#', '~'] else w for w in s.split(' ')) for s in summaries_raw]


 gil grissom entir team suspect bizarr cult murder evid tell disturb stori discov motiv massacr


In [43]:
compute_similarity('No more excus . a woman is talk to a man and he is smile TV : ryan stacey',
                   'kat persuad stacey and jean to come to R & R with kat and kim , leav charli and alfi to look after lili',
                    method=('tfidf', tfidf_weights)), compute_similarity('No more excus . a woman is talk to a man and he is smile TV : ryan stacey',
                   'kat persuad stacey and jean to come to R & R with kat and kim , leav charli and alfi to look after lili',
                    method=('count',))

(0, 3)

In [232]:
tfidf_weights['stacey']

3.337750494112952


CPU times: user 102 ms, sys: 3.63 ms, total: 105 ms
Wall time: 101 ms


In [68]:
print(len(summaries))
print(len(corpus))

41
51


In [71]:
best_matches_count[0]

(10.0, 43, 37)

In [106]:
for v, i, j in best_matches_count[:40]:
    print(i, df['content'].iloc[i])
    print("======================")
    #print(summaries_raw[j])
    #print("======================")
    #print(v)
    #print("======================")
    #print(list(w for w in set(summaries[j].split()) if w in corpus[i].split()))

43 [' ( In the garage , CATHERINE speaks into a tape recorder as she takes her notes . )', ' ( She looks at a schematic of the crime scene , # XR-0048-096822-003 . )', ' ( As she speaks , the camera follows on the map . )', " ( It starts with the figure of the father in the hallway in front of BRENDA 'S bedroom door ... )", ' [[Catherine]] notes on mapping : collins family murder', ' [[Catherine]] case # 00-398', ' [[Catherine]] intruder killed mother', ' ( ... then moves to the photo of the mother in the bedroom , Case # 00-398 , Photo # 17 . )', ' [[Catherine]] father went to protect the little girl', ' [[Catherine]] got nicked by knife', " ( Camera moves from the boy 's room on the map to the photo , Case # 00-398 , Photo # 47 . )", " [[Catherine]] killer subdued him as dad was entering the little girl 's room to", ' [[Catherine]] ... save her', " ( Something does n't click . )", " ( It does n't match the photos she 's looking at . )", ' ( She looks at the photo of the blood drops f

In [237]:
# similarity_matrix_tfidf = similarity_matrix

In [74]:
%%time
similarity_matrix_tfidf = np.zeros((len(df), len(summaries)))
for i, content in tqdm(enumerate(corpus), total=len(corpus)):
    for j, s in enumerate(summaries):
        similarity_matrix_tfidf[i, j] = compute_similarity(content, s, method=('tfidf_log_2', tfidf_weights))


CPU times: user 158 ms, sys: 248 µs, total: 158 ms
Wall time: 154 ms


In [75]:
np.max(similarity_matrix_tfidf)

5.073879393072383

In [76]:
similarity_matrix_tfidf.shape

(51, 41)

In [77]:
unrolled_sm = [(similarity_matrix_tfidf[i, j], i, j) for i in range(len(corpus)) for j in range(len(summaries))]
best_matches = sorted(unrolled_sm, key=lambda x: -x[0])

In [78]:
best_matches[:10]

[(5.073879393072383, 18, 11),
 (4.478210112207396, 30, 20),
 (4.4170448331849865, 34, 25),
 (4.086215946851232, 43, 38),
 (4.007664596229994, 43, 22),
 (3.752180905484431, 43, 37),
 (3.674869350872889, 18, 36),
 (3.6345832008706274, 43, 8),
 (3.419196111273055, 45, 37),
 (3.2263587192807943, 36, 27)]

In [375]:
for v, i, j in best_matches[:2]:
    print(i, df['content'].iloc[i])
    print("======================")
    #print(j, summaries_raw[j])
    print("======================")
    print()

KeyError: 'content'

# Making the cut

In [107]:
best_matches[:14]

[(5.073879393072383, 18, 11),
 (4.478210112207396, 30, 20),
 (4.4170448331849865, 34, 25),
 (4.086215946851232, 43, 38),
 (4.007664596229994, 43, 22),
 (3.752180905484431, 43, 37),
 (3.674869350872889, 18, 36),
 (3.6345832008706274, 43, 8),
 (3.419196111273055, 45, 37),
 (3.2263587192807943, 36, 27),
 (3.210115640509777, 16, 17),
 (3.143633682177115, 20, 12),
 (3.0970852254979633, 45, 38),
 (3.0310356319625127, 47, 36)]

[18, 30, 34, 43, 45, 36, 16, 20, 47, 12, 14, 17, 15, 33]

In [128]:

annotated_scenes

,scene_id,scene_text,in_summary,aspects,prediction
0,0,"["" ( Camera opens on the front of the Collins ...",1,Perpetrator,0
1,1,[' ( Emergency vehicles and Officer cars surro...,0,None,0
2,2,[' ( GRISSOM and SHIBLEY walk toward the open ...,0,None,0
3,3,"[' ( GRISSOM and SHIBELY walk inside . )', ' (...",1,Crime scene,0
4,4,[' ( The first thing they see down the hallway...,1,"Victim,Death cause",0
5,5,[' ( GRISSOM and SARA walk into the Master Bed...,1,"Victim,Death cause",0
6,6,[' ( SARA takes a picture of the dead woman . ...,1,Victim,0
7,7,"[' ( GRISSOM walks out of the house . )', "" ( ...",0,None,0
8,8,[' ( A reporter standing outside the house giv...,0,None,0
9,9,"["" ( GRISSOM walks up to DET O'RILEY who is ta...",0,None,0


In [129]:

f1_score(annotated_scenes.in_summary, annotated_scenes.prediction)

0.2857142857142857

In [252]:
votes_selection = {c:[] for c in ['Janine', 'Ryan', 'Stacey']}
votes_durations = {c:[] for c in ['Janine', 'Ryan', 'Stacey']}

for char in votes_selection:
    print(char.upper() + ' =====>')
    counter = 1
    for s in reses:
        if char in df['content'].iloc[s[0]]:
            print(df['content'].iloc[s[0]])
            votes_selection[char].append(df['sid'].iloc[s[0]])
            votes_durations[char].append(df['end'].iloc[s[0]] - df['begin'].iloc[s[0]])
        if len(votes_selection[char]) == 20: 
            break

JANINE =====>
Janine let you out then?  She doesn't own me. Good. I'm going down the Town Hall to  get Lily's birth certificate changed. Why today?  Cos I'm working after. Come on, you, me and Lily. It'll be fun.  All right?   Ryan Stacey
# Nothing more, nothing less  Love is the best... # You all right?  I know. Archie Mitchell's killer. I've  worked it out. Your precious Stacey.   Ryan Janine Stacey
Any time you need me, you know  where I am. Please, Janine. Please?  You've dropped your bombshell,  Pat, why don't you just fly off to New Zealand and enjoy  yourself with your real family. I don't know why I bother  caring about you.   Ryan Janine
I know you want nice little neat  answers. Life's not like that. Don't patronise me.  I'm not a kid any more. Then you should know you can't go  jumping to conclusions and calling it the truth cos that's  what you want to hear. Swear on my life  that it wasn't Stacey. I swear...  on your life...  that Stacey never killed  Archie Mitchell.   Ja

In [256]:
top_selection = {c:[] for c in ['Janine', 'Ryan', 'Stacey']}
top_durations = {c:[] for c in ['Janine', 'Ryan', 'Stacey']}
top_times     = {c:[] for c in ['Janine', 'Ryan', 'Stacey']}

for char in top_selection:
    print(char.upper() + ' =====>')
    counter = 1
    seen = set()
    for (v, s, l) in best_matches:
        if s not in seen and char in df['content'].iloc[s] + df['chars_query'].iloc[s]:
            sentence = df['transcript'].iloc[s].strip()
            print(sentence)
            
            top_selection[char].append(df['sid'].iloc[s])
            top_durations[char].append(round(df['end'].iloc[s] - df['begin'].iloc[s], 2))
            top_times[char].append((df['begin'].iloc[s], df['end'].iloc[s]))

            seen.add(s)
            while sentence and sentence[-1] not in ['.', '?', '!', "'"]:
                s += 1
                sentence = df['transcript'].iloc[s].strip()
                if sentence == '':
                    break
                print(' ++++ ', sentence)

                top_selection[char].append(df['sid'].iloc[s])
                top_durations[char].append(round(df['end'].iloc[s] - df['begin'].iloc[s], 2))
                top_times[char].append((df['begin'].iloc[s], df['end'].iloc[s]))
                seen.add(s)
                counter +=1
                
            counter += 1
        
        if len(seen) >= 20: 
            break

JANINE =====>
Janine let you out then?  She doesn't own me. Good. I'm going down the Town Hall to  get Lily's birth certificate changed. Why today?  Cos I'm working after. Come on, you, me and Lily. It'll be fun.  All right?
# Nothing more, nothing less  Love is the best... # You all right?  I know. Archie Mitchell's killer. I've  worked it out. Your precious Stacey.
Any time you need me, you know  where I am. Please, Janine. Please?  You've dropped your bombshell,  Pat, why don't you just fly off to New Zealand and enjoy  yourself with your real family. I don't know why I bother  caring about you.
I know you want nice little neat  answers. Life's not like that. Don't patronise me.  I'm not a kid any more. Then you should know you can't go  jumping to conclusions and calling it the truth cos that's  what you want to hear. Swear on my life  that it wasn't Stacey. I swear...  on your life...  that Stacey never killed  Archie Mitchell.
Look, whoever finds  the best Christmas present wins.

In [297]:
for char in top_selection:
    print(char.upper())
    for run in range(1, 5):
        print('Run', run, ':', ' '.join(sorted(top_selection[char][:5*run])))

JANINE
Run 1 : 176-1200 177-2024 178-51 182-14 182-1681
Run 2 : 176-1200 176-1785 176-2060 177-2024 178-51 181-116 182-14 182-1494 182-1681 183-1411
Run 3 : 175-2247 176-1200 176-1785 176-2060 177-2024 178-51 181-116 181-1544 182-1108 182-14 182-1494 182-1681 182-676 183-1411 185-992
Run 4 : 175-2247 176-1200 176-1785 176-2060 176-928 177-2024 178-51 181-116 181-1544 182-1108 182-14 182-1494 182-1681 182-676 183-1397 183-1411 183-1883 184-39 185-943 185-992
RYAN
Run 1 : 175-1772 176-1200 177-2024 178-1549 182-1681
Run 2 : 175-1772 176-1092 176-1200 177-2024 178-1549 179-1741 182-1203 182-1681 183-1411 185-438
Run 3 : 175-1772 176-1092 176-1200 176-2060 177-2024 178-1549 179-1741 179-940 181-1084 181-116 182-1203 182-1494 182-1681 183-1411 185-438
Run 4 : 175-1772 175-2247 176-1092 176-1200 176-1242 176-2060 177-2024 178-1549 179-1741 179-940 181-1084 181-116 181-1250 181-1544 182-1203 182-1279 182-1494 182-1681 183-1411 185-438
STACEY
Run 1 : 175-941 176-1200 177-2024 178-1549 185-316


In [298]:
for char in top_durations:
    print(char.upper())
    for run in range(1, 5):
        print('Run', run, ':', sum(top_durations[char][:5*run]))

JANINE
Run 1 : 70.37
Run 2 : 110.13000000000001
Run 3 : 128.97
Run 4 : 174.03
RYAN
Run 1 : 64.69
Run 2 : 121.7
Run 3 : 166.69
Run 4 : 198.14000000000001
STACEY
Run 1 : 52.66
Run 2 : 83.72999999999999
Run 3 : 126.74
Run 4 : 158.09999999999997


In [259]:
for char in votes_durations:
    print(char.upper())
    for run in range(1, 5):
        print('Run', run, ':', sum(votes_durations[char][:5*run]))

JANINE
Run 1 : 70.37000000000016
Run 2 : 110.1300000000018
Run 3 : 128.97000000000287
Run 4 : 174.03000000000267
RYAN
Run 1 : 64.69000000000051
Run 2 : 121.70000000000073
Run 3 : 166.69000000000148
Run 4 : 198.1400000000022
STACEY
Run 1 : 52.660000000000196
Run 2 : 93.0399999999998
Run 3 : 139.8
Run 4 : 162.10000000000065


In [291]:
top_times

{'Janine': [(3904.24, 3917.08),
  (7070.01, 7088.24),
  (5542.12, 5558.14),
  (238.06, 258.23),
  (37.09, 40.2),
  (4743.24, 4747.24),
  (5791.23, 5796.1),
  (4854.11, 4867.08),
  (347.02, 348.08),
  (6681.23, 6698.09),
  (6703.17, 6709.13),
  (5130.19, 5132.1),
  (2986.18, 2989.0),
  (2101.16, 2106.21),
  (3491.02, 3494.12),
  (2861.11, 2867.01),
  (6411.17, 6417.17),
  (4668.06, 4688.21),
  (2883.13, 2886.05),
  (138.01, 148.1)],
 'Ryan': [(3904.24, 3917.08),
  (7070.01, 7088.24),
  (5326.22, 5328.05),
  (5167.23, 5183.0),
  (5542.12, 5558.14),
  (3545.24, 3565.09),
  (1333.04, 1345.19),
  (5856.08, 5864.03),
  (3807.08, 3820.14),
  (4743.24, 4747.24),
  (3746.02, 3749.06),
  (4854.11, 4867.08),
  (3070.13, 3081.19),
  (347.02, 348.08),
  (6681.23, 6698.09),
  (4212.14, 4228.04),
  (6703.17, 6709.13),
  (4064.21, 4067.01),
  (4068.14, 4073.02),
  (5130.19, 5132.1)],
 'Stacey': [(2688.13, 2691.01),
  (3904.24, 3917.08),
  (883.21, 886.15),
  (7070.01, 7088.24),
  (5167.23, 5183.0),
  

In [261]:
pickle.dump(top_selection, open('selected_shots_final.pickle', 'wb'))
pickle.dump(top_times, open('selected_shots_boundaries_final.pickle', 'wb'))

# Generate the Videos!

In [281]:
! pip install -q moviepy

In [299]:
shot_boundaries = {}
for character in top_selection:
    for shot in top_selection[character]:
        if shot not in shot_boundaries:
            shot_boundaries[shot] = mastershot_df[mastershot_df.sid == shot][['start', 'end']].values[0]

In [300]:
set([k.split('-')[0] for k in shot_boundaries])

{'175', '176', '177', '178', '179', '181', '182', '183', '184', '185'}

In [301]:
shot_boundaries.keys()

dict_keys(['176-1200', '177-2024', '182-1681', '178-51', '182-14', '183-1411', '176-1785', '182-1494', '181-116', '176-2060', '175-2247', '181-1544', '185-992', '182-676', '182-1108', '176-928', '183-1883', '183-1397', '185-943', '184-39', '175-1772', '178-1549', '176-1092', '185-438', '179-1741', '182-1203', '181-1084', '179-940', '181-1250', '176-1242', '182-1279', '175-941', '185-316', '185-981', '184-2212', '184-2213', '182-1509', '182-1188', '177-1793', '182-57', '184-1785'])

In [ ]:
shot185_1736

In [284]:
3763.13 * 25

94078.25

In [285]:
os.listdir('../../eastenders_episodes')

['5531550228324592939.mp4',
 '5534228999422914578.mp4',
 '5542003749222140011.mp4',
 '5555325449284154780.mp4',
 '5547193787702629969.mp4',
 '5544620672795594434.mp4',
 '5555360238519252381.mp4',
 '5552368364300855101.mp4',
 '5549784941472309008.mp4',
 '5544574287152993687.mp4',
 '5539381671692122744.mp4']

In [286]:
"""%%time
frames = {}
# Read until video is completed
for filename in tqdm(os.listdir('../../eastenders_episodes'), total=11):
    print('Processing', filename)
    vid = cv2.VideoCapture('../../eastenders_episodes/' + filename)
    vid_id = file2id[filename]
    current_frame = 0
    
    for sid in sorted([sid for sid in shot_boundaries if sid.startswith(vid_id)]):
        start_frame = int(shot_boundaries[shot][0] * 25)
        end_frame = int(shot_boundaries[shot][1] * 25)
        frames[sid] = []
        
        while(cap.isOpened()):
            ret, frame = cap.read()
            current_frame += 1
            if ret == True and current_frame >= start_frame and current_frame <= end_frame:
                frames[sid].append(frame)
            if ret == True and current_frame > end_frame:
                print(f'frames[{sid}]:', len(frames[sid]))
                break

    vid.release()"""
print('Using OpenCV is too slow')

Using OpenCV is too slow


In [287]:
df[df.sid == '175-1772'][['begin', 'end']].values[0]

array([5326.22, 5328.05])

In [303]:
%%time
frames = {}
durs = {}
# Read until video is completed
for filename in tqdm(os.listdir('../../eastenders_episodes'), total=11):
    print('Processing', filename)
    video = VideoFileClip('../../eastenders_episodes/' + filename)
    vid_id = file2id[filename]
    current_frame = 0
    
    for sid in sorted([sid for sid in shot_boundaries if sid.startswith(vid_id)]):
        try:
            start, end = shot_boundaries[sid]
            frames[sid] = video.subclip(start, end)
            durs[sid] = (strptime(end, '%H:%M:%S.%f') - strptime(start, '%H:%M:%S.%f')).total_seconds()
        except Exception as e:
            print('EXCEPTION', str(e))
            print(sid, start, end)

Processing 5531550228324592939.mp4
Processing 5534228999422914578.mp4
Processing 5542003749222140011.mp4
Processing 5555325449284154780.mp4
Processing 5547193787702629969.mp4
Processing 5544620672795594434.mp4
Processing 5555360238519252381.mp4
Processing 5552368364300855101.mp4
Processing 5549784941472309008.mp4
Processing 5544574287152993687.mp4
Processing 5539381671692122744.mp4

CPU times: user 130 ms, sys: 4.15 s, total: 4.28 s
Wall time: 18.2 s


In [304]:
for char in top_selection:
    print(char.upper())
    print('Generating videos for', char, ':')
    for run in range(1, 5):
        t = time.time()
        print('   Run', run, '..', end=' ')
        shots = sorted(top_selection[char][:5*run])
        result = concatenate_videoclips([frames[sid] for sid in shots]) 
        result.write_videofile(f"videos/MeMAD_{run}_{char}.mp4",fps=25, logger=None)
        print(f'Done! ({(time.time() - t):.2f} s)')

JANINE
Generating videos for Janine :
   Run 1 .. Done! (20.18 s)
   Run 2 .. Done! (37.67 s)
   Run 3 .. Done! (45.12 s)
   Run 4 .. Done! (58.39 s)
RYAN
Generating videos for Ryan :
   Run 1 .. Done! (20.10 s)
   Run 2 .. Done! (37.25 s)
   Run 3 .. Done! (53.85 s)
   Run 4 .. Done! (68.36 s)
STACEY
Generating videos for Stacey :
   Run 1 .. Done! (20.21 s)
   Run 2 .. Done! (28.77 s)
   Run 3 .. Done! (44.57 s)
   Run 4 .. Done! (59.39 s)


In [351]:
results = {'Janine': {1:[('175_1', 5.3), ('175_2', 5.3)],
                      2:[('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3)],
                      3:[('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3), ('175_4', 5.3)],
                      4:[('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3), ('175_4', 5.3), ('175_5', 5.3)],},
           'Ryan':   {1:[('175_1', 5.3), ('175_2', 5.3)],
                      2:[('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3)],
                      3:[('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3), ('175_4', 5.3)],
                      4:[('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3), ('175_4', 5.3), ('175_5', 5.3)],},
           'Stacey': {1:[('175_1', 5.3), ('175_2', 5.3)],
                      2:[('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3)],
                      3:[('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3), ('175_4', 5.3)],
                      4:[('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3), ('175_4', 5.3), ('175_5', 5.3)],}}

In [354]:
results

{'Janine': {1: [('175_1', 5.3), ('175_2', 5.3)],
  2: [('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3)],
  3: [('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3), ('175_4', 5.3)],
  4: [('175_1', 5.3),
   ('175_2', 5.3),
   ('175_3', 5.3),
   ('175_4', 5.3),
   ('175_5', 5.3)]},
 'Ryan': {1: [('175_1', 5.3), ('175_2', 5.3)],
  2: [('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3)],
  3: [('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3), ('175_4', 5.3)],
  4: [('175_1', 5.3),
   ('175_2', 5.3),
   ('175_3', 5.3),
   ('175_4', 5.3),
   ('175_5', 5.3)]},
 'Stacey': {1: [('175_1', 5.3), ('175_2', 5.3)],
  2: [('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3)],
  3: [('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3), ('175_4', 5.3)],
  4: [('175_1', 5.3),
   ('175_2', 5.3),
   ('175_3', 5.3),
   ('175_4', 5.3),
   ('175_5', 5.3)]}}

In [367]:
results = {}
for char in top_selection:
    results[char] = {}
    for run in range(1, 5):
        shots = sorted(top_selection[char][:5*run])
        results[char][run] = [(shot.replace('-', '_'), durs[shot]) for shot in shots] # [('100-1', 10.5) for shot in shots] #

In [368]:
results

{'Janine': {1: [('176_1200', 12.84),
   ('177_2024', 18.23),
   ('178_51', 20.17),
   ('182_14', 3.11),
   ('182_1681', 16.02)],
  2: [('176_1200', 12.84),
   ('176_1785', 4.87),
   ('176_2060', 16.86),
   ('177_2024', 18.23),
   ('178_51', 20.17),
   ('181_116', 1.06),
   ('182_14', 3.11),
   ('182_1494', 12.97),
   ('182_1681', 16.02),
   ('183_1411', 4.0)],
  3: [('175_2247', 5.96),
   ('176_1200', 12.84),
   ('176_1785', 4.87),
   ('176_2060', 16.86),
   ('177_2024', 18.23),
   ('178_51', 20.17),
   ('181_116', 1.06),
   ('181_1544', 1.91),
   ('182_1108', 3.1),
   ('182_14', 3.11),
   ('182_1494', 12.97),
   ('182_1681', 16.02),
   ('182_676', 5.05),
   ('183_1411', 4.0),
   ('185_992', 2.82)],
  4: [('175_2247', 5.96),
   ('176_1200', 12.84),
   ('176_1785', 4.87),
   ('176_2060', 16.86),
   ('176_928', 5.9),
   ('177_2024', 18.23),
   ('178_51', 20.17),
   ('181_116', 1.06),
   ('181_1544', 1.91),
   ('182_1108', 3.1),
   ('182_14', 3.11),
   ('182_1494', 12.97),
   ('182_1681',

In [376]:
def generate_results(results):
    team_name = 'MeMAD'
    desc = "The output"
    query_characters = ['Janine', 'Ryan', 'Stacey']
    shots_per_run = {1: 5, 2: 10, 3:15, 4:20}

    for run in [1, 2, 3, 4]:
        with open(f'xml/{team_name}_run_{run}.xml', 'wt') as file:
            file.write('<!DOCTYPE videoSummarizationResults SYSTEM "https://www-nlpir.nist.gov/projects/tv2020/dtds/videoSummarizationResults.dtd">\n')
            file.write('<videoSummarizationResults>\n')
            file.write(f'\t<videoSummarizationRunResult pid="{team_name}" priority="{run}" desc="Run {run}">\n')
            for character in query_characters:
                n_shots = shots_per_run[run]
                duration = sum([c[1] for c in results[character][run]])
                file.write(f'\t\t<videoSummarizationTopicResult  target="{character}" numShots="{shots_per_run[run]}" summTime="{round(duration, 1)}">\n')
                for i, shot_id in enumerate(results[character][run]):
                    file.write(f'\t\t\t<item seqNum="{i+1}" shotId="shot{shot_id[0]}"/>\n')
                file.write(f'\t\t</videoSummarizationTopicResult>\n')
            file.write('\t</videoSummarizationRunResult>\n')
            file.write('</videoSummarizationResults>\n')

In [377]:
generate_results(results)